# TP 06 : Mini-GPT Partie 2 - Fine-tuning GPT-2

**Objectif** : Fine-tuner GPT-2 français pour générer des descriptions de Pokémon

**Programme** :
1. Rappel : Ce qu'on a fait avec les noms
2. Démo comparative : From scratch vs Fine-tuning
3. Introduction au Transfer Learning
4. Charger GPT-2 français
5. Préparer le dataset Pokémon
6. Fine-tuning sur Colab
7. Génération de descriptions
8. Pipeline complet : Nom + Description
9. Démo collective : Inventez votre Pokémon !

**Durée** : 2h

---

## 0. Installation et imports

In [ ]:
# Installation des dépendances (Colab)
!pip install -q transformers datasets accelerate

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
import random
import warnings
warnings.filterwarnings('ignore')

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Reproductibilité
torch.manual_seed(42)
random.seed(42)

---

## 1. Rappel : Ce qu'on a fait avec les noms

En Session 5, nous avons :
- Construit un **Mini-GPT from scratch** (~500K paramètres)
- Entraîné sur **1211 noms de Pokémon**
- Généré des noms comme "Pikadon", "Salamèdre", "Florizard"

### Limites du modèle from scratch

| Aspect | From Scratch | Pré-entraîné |
|--------|--------------|---------------|
| Données nécessaires | Beaucoup | Peu (fine-tuning) |
| Temps d'entraînement | Long | Court |
| Qualité du langage | Limitée | Excellente |
| Vocabulaire | Restreint | Complet |

**Question** : Et si on voulait générer des **descriptions complètes** de Pokémon ?

---

## 2. Démo comparative : From scratch vs Fine-tuning

Imaginons entraîner un modèle from scratch sur notre corpus Pokémon (~69 MB de texte).

### Problèmes :
1. **Temps** : Des heures/jours d'entraînement
2. **Ressources** : GPU puissant nécessaire
3. **Données** : 69 MB = petit pour un LLM
4. **Qualité** : Le modèle doit tout apprendre (grammaire, syntaxe, sémantique...)

### Solution : Transfer Learning !

```
Modèle pré-entraîné (GPT-2)     Notre corpus Pokémon
        ↓                              ↓
   Connaissance                    Spécialisation
   du français                     Pokémon
        ↓                              ↓
        └──────────┬───────────────────┘
                   ↓
          Modèle fine-tuné
          (Expert Pokémon)
```

---

## 3. Introduction au Transfer Learning

### Qu'est-ce que le Transfer Learning ?

**Définition** : Utiliser un modèle pré-entraîné sur une grande quantité de données, puis l'adapter à une tâche spécifique.

### Analogie

- **From scratch** = Apprendre le français à un bébé, puis lui enseigner les Pokémon
- **Fine-tuning** = Prendre un adulte francophone et lui montrer des articles Pokémon

### Avantages du fine-tuning

1. **Rapidité** : Quelques minutes au lieu d'heures
2. **Efficacité** : Peu de données suffisent
3. **Qualité** : Le modèle sait déjà écrire en français
4. **Coût** : Moins de ressources GPU

### Types de fine-tuning

| Type | Description | Usage |
|------|-------------|-------|
| **Full fine-tuning** | Tous les poids sont mis à jour | Petit modèle, beaucoup de données |
| **LoRA** | Seuls quelques poids sont ajoutés | Grand modèle, peu de ressources |
| **Prompt tuning** | Seul le prompt est optimisé | Très peu de données |

Aujourd'hui, nous ferons du **full fine-tuning** sur un petit GPT-2.

---

## 4. Charger GPT-2 français

Nous utilisons `asi/gpt-fr-cased-small` : un GPT-2 entraîné sur du texte français.

### Caractéristiques
- **~117M paramètres** (vs 1.5B pour GPT-2 XL)
- **Vocabulaire français** optimisé
- **Licence** : Open source

In [ ]:
# Charger le tokenizer et le modèle
model_name = "asi/gpt-fr-cased-small"

print("Chargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Chargement du modèle...")
model = AutoModelForCausalLM.from_pretrained(model_name)

print(f"\n✅ Modèle chargé !")
print(f"Paramètres : {sum(p.numel() for p in model.parameters()):,}")
print(f"Vocabulaire : {len(tokenizer):,} tokens")

In [ ]:
# Ajouter un token de padding (nécessaire pour le fine-tuning)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

### Test du modèle avant fine-tuning

Que sait GPT-2 français des Pokémon ?

In [ ]:
def generate_text(prompt, max_length=100, temperature=0.8, top_k=50):
    """Génère du texte à partir d'un prompt."""
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Déplacer le modèle sur GPU si disponible
model = model.to(device)

In [ ]:
# Test avant fine-tuning
prompt = "Pikachu est un Pokémon de type"
print(f"Prompt: {prompt}")
print(f"Génération (AVANT fine-tuning):")
print(generate_text(prompt))

**Observation** : Le modèle peut générer du français correct, mais ne connaît pas vraiment les Pokémon. C'est là qu'intervient le fine-tuning !

---

## 5. Préparer le dataset Pokémon

Notre corpus : `chris-lmd/pokepedia-fr` (28,677 articles, ~69 MB)

In [ ]:
# Charger le dataset
print("Chargement du dataset Pokémon...")
dataset = load_dataset("chris-lmd/pokepedia-fr")

print(f"\n✅ Dataset chargé !")
print(f"Articles : {len(dataset['train']):,}")
print(f"Colonnes : {dataset['train'].column_names}")

In [ ]:
# Aperçu d'un article
sample = dataset['train'][0]
print(f"Titre: {sample.get('title', 'N/A')}")
print(f"\nContenu (500 premiers caractères):")
print(sample['text'][:500] + "...")

### Filtrer les articles pertinents

On garde les articles qui décrivent des Pokémon (pas les pages techniques, listes, etc.)

In [ ]:
def is_pokemon_description(example):
    """Filtre les articles décrivant des Pokémon."""
    text = example['text'].lower()
    # Cherche des patterns typiques de descriptions de Pokémon
    pokemon_patterns = [
        "est un pokémon",
        "type ",
        "évolution",
        "espèce",
        "capacité",
        "génération"
    ]
    return any(pattern in text for pattern in pokemon_patterns)

# Filtrer
filtered_dataset = dataset['train'].filter(is_pokemon_description)
print(f"Articles après filtrage : {len(filtered_dataset):,}")

In [ ]:
# Pour le fine-tuning, on limite à un sous-ensemble (vitesse)
# En production, on utiliserait tout le dataset
MAX_SAMPLES = 2000

if len(filtered_dataset) > MAX_SAMPLES:
    train_dataset = filtered_dataset.shuffle(seed=42).select(range(MAX_SAMPLES))
else:
    train_dataset = filtered_dataset

print(f"Échantillon pour fine-tuning : {len(train_dataset):,} articles")

### Tokenization du dataset

In [ ]:
# Configuration
MAX_LENGTH = 256  # Longueur max des séquences

def tokenize_function(examples):
    """Tokenize les textes avec troncature."""
    return tokenizer(
        examples['text'],
        truncation=True,
        max_length=MAX_LENGTH,
        padding='max_length'
    )

# Tokenizer le dataset
print("Tokenization en cours...")
tokenized_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

print(f"✅ Tokenization terminée !")

In [ ]:
# Pour le language modeling, labels = input_ids
def add_labels(examples):
    examples['labels'] = examples['input_ids'].copy()
    return examples

tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)
print(f"Colonnes finales : {tokenized_dataset.column_names}")

---

## 6. Fine-tuning sur Colab

On utilise le `Trainer` de Hugging Face pour simplifier l'entraînement.

In [ ]:
# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="./gpt2-pokemon",
    overwrite_output_dir=True,
    
    # Hyperparamètres
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # Effective batch size = 16
    
    # Optimiseur
    learning_rate=5e-5,
    warmup_steps=100,
    weight_decay=0.01,
    
    # Logging
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    
    # Misc
    fp16=torch.cuda.is_available(),  # Mixed precision si GPU
    report_to="none",  # Désactiver wandb
)

print("Configuration :")
print(f"  - Epochs : {training_args.num_train_epochs}")
print(f"  - Batch size effectif : {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  - Learning rate : {training_args.learning_rate}")
print(f"  - FP16 : {training_args.fp16}")

In [ ]:
# Créer le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

print("✅ Trainer créé !")

In [ ]:
# Lancer le fine-tuning
print("🚀 Fine-tuning en cours...")
print("(Cela prend environ 10-15 minutes sur GPU Colab)")
print()

trainer.train()

print("\n✅ Fine-tuning terminé !")

In [ ]:
# Sauvegarder le modèle
trainer.save_model("./gpt2-pokemon-final")
tokenizer.save_pretrained("./gpt2-pokemon-final")
print("✅ Modèle sauvegardé !")

---

## 7. Génération de descriptions

Testons notre modèle fine-tuné !

In [ ]:
# Recharger le modèle fine-tuné (optionnel, déjà en mémoire)
# model = AutoModelForCausalLM.from_pretrained("./gpt2-pokemon-final").to(device)

In [ ]:
def generate_description(prompt, max_length=150, temperature=0.8, top_k=50, top_p=0.9):
    """Génère une description de Pokémon."""
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            repetition_penalty=1.2,  # Évite les répétitions
            no_repeat_ngram_size=3
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Test 1 : Pikachu
prompt = "Pikachu est un Pokémon de type"
print(f"Prompt: {prompt}")
print(f"\nGénération (APRÈS fine-tuning):")
print(generate_description(prompt))

In [ ]:
# Test 2 : Pokémon inventé
prompt = "Flamador est un Pokémon de type Feu. Il"
print(f"Prompt: {prompt}")
print(f"\nGénération:")
print(generate_description(prompt))

In [ ]:
# Test 3 : Évolution
prompt = "Aquazur évolue en"
print(f"Prompt: {prompt}")
print(f"\nGénération:")
print(generate_description(prompt))

### Explorer les paramètres de génération

In [ ]:
prompt = "Voltorex est un Pokémon de type Électrik. Ses capacités incluent"

print("=" * 60)
print("Effet de la température")
print("=" * 60)

for temp in [0.3, 0.7, 1.0, 1.3]:
    print(f"\n🌡️ Température = {temp}")
    print(generate_description(prompt, temperature=temp, max_length=100))
    print("-" * 40)

**Observation** :
- **Température basse (0.3)** : Texte répétitif mais cohérent
- **Température moyenne (0.7)** : Bon équilibre
- **Température haute (1.3)** : Plus créatif mais parfois incohérent

---

## 8. Pipeline complet : Nom + Description

Combinons notre Mini-GPT (noms) et GPT-2 fine-tuné (descriptions) !

In [ ]:
# Charger le modèle de noms (depuis la session 5)
# Si vous avez sauvegardé le modèle, chargez-le ici
# Sinon, on utilise une version simplifiée

NOMS_POKEMON_EXEMPLES = [
    "Flamidor", "Aquazur", "Voltorex", "Plantix", "Glacior",
    "Pyralis", "Hydragon", "Électrus", "Floralix", "Givreon",
    "Magmador", "Ondiral", "Fulguras", "Sylvora", "Crystallix",
    "Embrasol", "Tsunamix", "Zappeur", "Verdora", "Frostix"
]

TYPES_POKEMON = [
    "Feu", "Eau", "Électrik", "Plante", "Glace",
    "Combat", "Poison", "Sol", "Vol", "Psy",
    "Insecte", "Roche", "Spectre", "Dragon", "Ténèbres",
    "Acier", "Fée", "Normal"
]

def generate_name():
    """Génère un nom de Pokémon (version simplifiée)."""
    # En production, utiliser le Mini-GPT de la session 5
    return random.choice(NOMS_POKEMON_EXEMPLES)

In [ ]:
def create_pokemon():
    """Pipeline complet : génère un Pokémon avec nom et description."""
    # 1. Générer un nom
    nom = generate_name()
    
    # 2. Choisir un type
    type_pokemon = random.choice(TYPES_POKEMON)
    
    # 3. Générer la description
    prompt = f"{nom} est un Pokémon de type {type_pokemon}."
    description = generate_description(prompt, max_length=200, temperature=0.8)
    
    return {
        "nom": nom,
        "type": type_pokemon,
        "description": description
    }

In [ ]:
# Générer un Pokémon complet
pokemon = create_pokemon()

print("🎮 NOUVEAU POKÉMON GÉNÉRÉ !")
print("=" * 50)
print(f"📛 Nom : {pokemon['nom']}")
print(f"⚡ Type : {pokemon['type']}")
print(f"\n📝 Description :")
print(pokemon['description'])

### Version améliorée avec le Mini-GPT

In [ ]:
# Si vous avez sauvegardé votre Mini-GPT de la session 5 :
# Décommentez et adaptez le chemin

"""
from minigpt_session5 import MiniGPT, CharTokenizer

# Charger le modèle de noms
char_tokenizer = CharTokenizer()
minigpt = MiniGPT(vocab_size=len(char_tokenizer), ...)
minigpt.load_state_dict(torch.load("minigpt_pokemon.pt"))
minigpt.eval()

def generate_name_with_minigpt(prefix=""):
    # Génération avec le vrai Mini-GPT
    return minigpt.generate(prefix, max_length=15)
"""
print("💡 Pour utiliser le vrai Mini-GPT, chargez le modèle de la session 5")

---

## 9. Démo collective : Inventez votre Pokémon !

Chaque élève peut maintenant créer son propre Pokémon unique !

In [ ]:
def inventez_votre_pokemon(prefixe_nom=None, type_souhaite=None):
    """
    Créez votre propre Pokémon !
    
    Args:
        prefixe_nom: Début du nom (ex: "Pika", "Dra", "Fla")
        type_souhaite: Type voulu (ex: "Feu", "Eau", "Dragon")
    """
    # Nom
    if prefixe_nom:
        # Compléter le préfixe (version simplifiée)
        suffixes = ["dor", "rex", "ix", "on", "us", "al", "or", "ax", "is", "eon"]
        nom = prefixe_nom.capitalize() + random.choice(suffixes)
    else:
        nom = generate_name()
    
    # Type
    type_pokemon = type_souhaite if type_souhaite else random.choice(TYPES_POKEMON)
    
    # Description
    prompt = f"{nom} est un Pokémon de type {type_pokemon}."
    description = generate_description(prompt, max_length=200, temperature=0.8)
    
    # Affichage stylé
    print("\n" + "═" * 60)
    print("          🎮 VOTRE POKÉMON UNIQUE ! 🎮")
    print("═" * 60)
    print(f"\n  📛 Nom : {nom}")
    print(f"  ⚡ Type : {type_pokemon}")
    print(f"\n  📝 Description :")
    print(f"  {description}")
    print("\n" + "═" * 60)
    
    return {"nom": nom, "type": type_pokemon, "description": description}

In [ ]:
# Exemple 1 : Pokémon totalement aléatoire
inventez_votre_pokemon()

In [ ]:
# Exemple 2 : Avec un préfixe de nom
inventez_votre_pokemon(prefixe_nom="Pyro")

In [ ]:
# Exemple 3 : Avec préfixe et type
inventez_votre_pokemon(prefixe_nom="Crystal", type_souhaite="Glace")

In [ ]:
# 🎯 À VOUS DE JOUER !
# Remplacez les valeurs ci-dessous pour créer VOTRE Pokémon

inventez_votre_pokemon(
    prefixe_nom="",      # Votre préfixe (ex: "Méga", "Ultra", votre prénom...)
    type_souhaite=""     # Votre type (ex: "Dragon", "Fée", "Spectre"...)
)

---

## 10. Récapitulatif

### Ce que vous avez appris

1. **Transfer Learning** : Pourquoi c'est plus efficace que from scratch
2. **Fine-tuning** : Adapter un modèle pré-entraîné à une tâche
3. **Hugging Face** : Charger modèles et tokenizers
4. **Trainer API** : Entraîner facilement avec PyTorch
5. **Génération** : Paramètres (temperature, top_k, top_p)
6. **Pipeline ML** : Combiner plusieurs modèles

### Architecture du projet

```
┌─────────────────────────────────────────────────────────────┐
│                    PIPELINE POKÉMON                         │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│    ┌─────────────────┐     ┌─────────────────────────┐     │
│    │   Mini-GPT      │     │    GPT-2 Fine-tuné      │     │
│    │  (from scratch) │     │    (Transfer Learning)  │     │
│    │                 │     │                         │     │
│    │  Génère des     │────▶│  Génère des             │     │
│    │  NOMS           │     │  DESCRIPTIONS           │     │
│    │                 │     │                         │     │
│    │  ~500K params   │     │  ~117M params           │     │
│    └─────────────────┘     └─────────────────────────┘     │
│                                                             │
│                           ↓                                 │
│                                                             │
│               ┌───────────────────────┐                     │
│               │   Pokémon Complet !   │                     │
│               │                       │                     │
│               │  Nom + Type +         │                     │
│               │  Description          │                     │
│               └───────────────────────┘                     │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### Pour aller plus loin

- **LoRA** : Fine-tuning efficace pour grands modèles
- **RLHF** : Reinforcement Learning from Human Feedback
- **RAG** : Retrieval Augmented Generation
- **Modèles multimodaux** : Génération image + texte

---

## 🎉 Félicitations !

Vous avez maintenant :
- Compris les **Transformers** de A à Z
- Implémenté l'**Attention**, **Multi-Head**, et un **Transformer Block**
- Créé un **Mini-GPT from scratch**
- **Fine-tuné GPT-2** sur un corpus personnalisé
- Construit un **pipeline de génération** complet

**Vous êtes prêts pour le monde des LLM !** 🚀